In [44]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import requests
from pandas.io.json import json_normalize

# Hello Capstone Project Course!

I will mostly be using this notebook for my capstone project with Coursera

I will peridoically update it

I have saved the data from the scraping and cleaning into a csv file, and I load it in the following cell

In [45]:
df=pd.read_csv('lat_lon_ata.csv').iloc[:,1:]

In [46]:
df.head()


,Postcode,Borough,Neighbourhood,Lat,Long
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763


I have saved my foursquare credentials into an external text file, I load those credentials in the following cell

In [47]:
with open('credentials.txt') as creds:
    lines=creds.readlines()
    
for i in range(len(lines)):
    lines[i]=lines[i].strip('\n')

In [48]:
CLIENT_ID,CLIENT_SECRET,VERSION,LIMIT=lines

In [142]:
search_query = 'Park Golf'
radius = 1000

In [143]:
urls=[]
for i in list(df.index):
    lat,long=list(df.iloc[i].values[-2:])
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, long, VERSION, search_query, radius, LIMIT)
    urls.append(url)

The following two cells use code given in the Four Square API tutorial, I modified it slightly to better suit this project, but it is largely identical.

In [144]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [166]:
dfs_all=[]
features=[]
for i in urls[:4]:
    results=requests.get(i).json()
    try:
        venues = results['response']['venues']
        res_df = json_normalize(venues)
        filtered_columns = ['name', 'categories'] + [col for col in res_df.columns if col.startswith('location.')] + ['id']
        dataframe_filtered = res_df.loc[:, filtered_columns]
        dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)
        dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
        dataframe_filtered_p=dataframe_filtered[dataframe_filtered['categories']=='Park']
        dataframe_filtered_g=dataframe_filtered[dataframe_filtered['categories']=='Golf Course']
        dfs_all.append(dataframe_filtered)
        feat=[len(list(dataframe_filtered_p.index)),
                         dataframe_filtered_p['distance'].mean(),
                         len(list(dataframe_filtered_g.index)),
                         dataframe_filtered_g['distance'].mean()]
        for i in range(2):
            if feat[2*i]==0:
                feat[2*i+1]=0
        features.append(feat)
    except KeyError:
        features.append([0,
                         0,
                         0,
                         0])
    
    
    

In [135]:
dfs_all[1]['distance'].mean()

740.5

In [136]:
dfs_all[0]

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
1,Brookbanks Park,Park,Toronto,CA,Toronto,Canada,NaN,245,"[Toronto, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.75197604605557...",43.751976,-79.332140,NaN,ON,4e8d9dcdd5fbbbb6b3003c7b
7,Ranchdale Park,Park,NaN,CA,NaN,Canada,NaN,639,[Canada],"[{'label': 'display', 'lat': 43.751388, 'lng':...",43.751388,-79.322138,NaN,NaN,4fad6522e4b08da80a56db50
8,Three Valleys Park,Park,NaN,CA,NaN,Canada,NaN,660,[Canada],"[{'label': 'display', 'lat': 43.75119469806736...",43.751195,-79.337356,NaN,NaN,53695918498e12363ff0d9ad
10,Sandover Park,Park,Sandover Dr.,CA,Toronto,Canada,at Clayland Dr.,834,"[Sandover Dr. (at Clayland Dr.), Toronto ON, C...","[{'label': 'display', 'lat': 43.76027747979718...",43.760277,-79.333305,NaN,ON,4de4dc58e4cd056f747bbaef
